In [ ]:
# !ssh root@ssh.openbayes.com -p32599 /etc/welcome
# !ssh root@ssh.openbayes.com -p32599 /etc/autodl-motd
import os

ssh_cmd = 'ssh root@ssh.openbayes.com -p32599'

root_dir = '/openbayes/home/'

comfyUI_dir = os.path.join(root_dir, './ComfyUI/')
sdwebUI_dir = os.path.join(root_dir, './stable-diffusion-webui/')

print(comfyUI_dir)

def check_remote_folder_exists(ssh_cmd, remote_dir):
    cmd = ssh_cmd + ' ls ' + remote_dir
    print(cmd)
    result = os.system(cmd)
    if result == 0:
        # print('Folder exists')
        return True
    else:
        # print('Folder not exists')
        return False

if check_remote_folder_exists(ssh_cmd, comfyUI_dir):
    repo_dir = comfyUI_dir
    remote_VAE_dir = os.path.join(repo_dir, './models/vae/')
    remote_lora_dir = os.path.join(repo_dir, './models/loras/')
    remote_sd_dir = os.path.join(repo_dir, './models/checkpoints/')
    remote_embeddings_dir = os.path.join(repo_dir, './models/embeddings/')
    print('ComfyUI repo found')
elif check_remote_folder_exists(ssh_cmd, sdwebUI_dir):
    repo_dir = sdwebUI_dir
    remote_VAE_dir = os.path.join(repo_dir, './models/VAE/')
    remote_lora_dir = os.path.join(repo_dir, './models/Lora/')
    remote_sd_dir = os.path.join(repo_dir, './models/Stable-diffusion/')
    remote_embeddings_dir = os.path.join(repo_dir, './embeddings/')
    print('Stable-diffusion repo found')
else:
    print('No repo found')

local_dir = "F:/AI_research/Stable_Diffusion/stable-diffusion-webui-master/models/"

In [ ]:
import os

def generate_file_dict(local_dir, mappings):
    file_dict = {}
    for root, dirs, files in os.walk(local_dir):
        for filename in files:
            local_path = os.path.normpath(os.path.join(root, filename))
            for key, value in mappings.items():
                if filename == key:
                    remote_path = value
                    file_dict[local_path] = remote_path
    return file_dict

# 创建三个字典
vae_keys = [
    "orangemix.vae.pt",
    "kl-f8-anime2.ckpt",
    "anything-v4.0.vae.pt",
    "vae-ft-mse-840000-ema-pruned.ckpt"
]
vae_dict = {key: remote_VAE_dir for key in vae_keys}

lora_keys = [
    "epiNoiseoffset_v2.safetensors",
    "epiNoiseoffset_v2Pynoise.safetensors",
    "animeLineartStyle_v20Offset.safetensors",
    "tokiameStyleThick.safetensors",
    "thickerLinesAnimeStyle_loraVersion.safetensors"
]
lora_dict = {key: remote_lora_dir for key in lora_keys}

sd_keys = [
    "expmixLine_v20.safetensors",
    "camelliamixLine_v10.safetensors",
    "chilloutmix_NiPrunedFp16.safetensors",
    "PVC-AO.safetensors",
    "midjourneyPapercut_v1.ckpt",
    "chilloutmix_NiPrunedFp16Fix.safetensors",
    "anything-v4.5-pruned.safetensors",
    "anything-v4.5-pruned-fp16.ckpt"
]
sd_dict = {key: remote_sd_dir for key in sd_keys}

# 将三个字典合并
sum_mappings = {**vae_dict, **lora_dict, **sd_dict}

file_dict = generate_file_dict(local_dir, sum_mappings)

print(file_dict)

In [ ]:
import os

def batch_scp(file_dict, username, remote_host, port):
    for local_path, remote_path in file_dict.items():
        # 检查远程路径下是否存在同名文件
        local_filename = os.path.basename(local_path)
        check_cmd = f'ssh {username}@{remote_host} -p{port} find {remote_path} -name {local_filename}' 
        result = !{check_cmd}
        print(result)
        cmd = f'scp -P {port} {local_path} {username}@{remote_host}:{remote_path}'
        if not result:
            # 传输文件到远程主机
            print(f'❓{local_filename}')
            print(cmd)
            !$cmd
            # print(f"Transferred {local_filename} to {remote_path}")
        else:
            # print(f"File {local_filename} already exists on remote host, skipping transfer...")
            print(f'✅{local_filename}')
            pass

def extract_ssh_info(ssh_cmd):
    """
    从SSH命令中提取用户名、远程主机名和端口号。

    Args:
        ssh_cmd (str): SSH命令字符串。

    Returns:
        tuple: 包含用户名、远程主机名和端口号的元组。
    """
    # 分离用户名和主机名
    username, _, hostname = ssh_cmd.split()[1].partition('@')
    # 获取端口号
    port = ssh_cmd.split('-p')[-1].split()[0]
    return username, hostname, port


ssh_cmd = 'ssh root@ssh.openbayes.com -p32599'
username, remote_host, port = extract_ssh_info(ssh_cmd)

batch_scp(file_dict, username, remote_host, port)

